In [11]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [12]:
# The working notebook file path is stored in NOTEBOOK_FULL_PATH
# Change working directory to project root which is parent of NOTEBOOK_FULL_PATH
# Typing Shift + Enter on obove cell to help proper javascript execution

import os
import time
from pathlib import Path
from jupyter_core.paths import jupyter_config_dir
from traitlets.config import Config

def get_config():
  return Config()

config_file = os.path.join(jupyter_config_dir(), 'jupyter_notebook_config.py')
config_content = open(config_file).read()
c = get_config()
exec(config_content)
root_dir = c['ContentsManager']['root_dir']
project_root = Path(root_dir + NOTEBOOK_FULL_PATH).parent
print(f'Working directory has been changed to {project_root}')
os.chdir(project_root)
os.listdir('./')

Working directory has been changed to /workspace/qna-assistant


['plan_and_execute_agent__2nb__.ipynb',
 '.env',
 '.git',
 '__pycache__',
 'askLangchain__2nb__.md',
 'typesense-data',
 'askLangchain__2nb__.ipynb',
 'langchain_doc_sitemap.py',
 'tmp',
 'docker-compose.yml',
 'search.py',
 '.gitignore',
 'plan_and_execute_agent__2nb__.md',
 'my-crawler',
 'tags',
 'requirements.txt',
 '.ipynb_checkpoints']

In [13]:
from dotenv import load_dotenv
from typing import Optional, Type
import arrow

from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.tools.base import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper


load_dotenv()

True

In [14]:
class CurrentDateTimeIndicator(BaseTool):
    name = 'current_date_time'
    description = "Useful for when you need to know current date or current time"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        utc = arrow.utcnow()
        return utc.format('HH:mm:ss dddd, MMMM D, YYYY')

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

current_date_time_tool = CurrentDateTimeIndicator()
current_date_time_tool.run(tool_input={'query': None})

'09:59:19 Saturday, July 22, 2023'

In [15]:
search = DuckDuckGoSearchAPIWrapper()
search.run("Leo DiCaprio's girlfriend current age")

"Leonardo DiCaprio started dating hot 25 year olds back in the roaring '90s and...truly nothing has changed! Other than the people he's dating, of course, because lord knows this man won't stay... The actor made headlines last year for breaking up with his 25-year-old girlfriend Camila Morrone, further fuelling the theory that he does not date any women over that age. But now, the Hollywood star reportedly appears to be reaching even younger, having been linked to a 19-year-old Israeli model, Eden Polani. Special Series Main Character of the Day Leo DiCaprio's dating history is part of our obsession with staying young forever February 9, 20234:58 PM ET Manuela López Restrepo Enlarge this image... Whether or not that's true, it's rumored that his latest girlfriend is Israeli model, 19-year-old Eden Polani. Main image, Leonardo DiCaprio has become the subject of jokes on social media... The 25-year-old age limit was again brought up last year when he split from model Camila Morrone short

In [16]:
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    current_date_time_tool,
]
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [17]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new PlanAndExecute chain...
steps=[Step(value="Search for information about Leo DiCaprio's girlfriend."), Step(value='Find her current age.'), Step(value='Calculate her current age raised to the 0.43 power.'), Step(value="Given the above steps taken, respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Action:
{
  "action": "Search",
  "action_input": "Leo DiCaprio girlfriend"
}

> Finished chain.
*****

Step: Search for information about Leo DiCaprio's girlfriend.

Response: Action:
{
  "action": "Search",
  "action_input": "Leo DiCaprio girlfriend"
}

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search",
  "action_input": "Leo DiCaprio girlfriend age"
}
```
Observation: Jeff J Mitchell/Getty Images Welcome to a new NPR series where we spotlight the people and things making headlines — and the stories behind them. The 48-year-old generated outrage online when it... They were first spotted together in 2017 and little ha

"Camila Morrone's current age is 25. Her age raised to the power of 0.43 is approximately 3.991298452658078."

In [18]:
agent.run("How many years from now Qatar worldcup happened")



> Entering new PlanAndExecute chain...
steps=[Step(value='Determine the current year.'), Step(value='Determine the year in which the Qatar World Cup took place.'), Step(value='Calculate the difference between the current year and the year of the Qatar World Cup.'), Step(value="Given the above steps taken, respond to the user's original question. \n\n")]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "current_date_time",
  "action_input": "{\"query\": \"current year\"}"
}
```
Observation: 09:59:38 Saturday, July 22, 2023
Thought:I need to determine the current year.

> Finished chain.
*****

Step: Determine the current year.

Response: I need to determine the current year.

> Entering new AgentExecutor chain...
Thought: I need to search for information about the year in which the Qatar World Cup took place. I can use the search tool to find the answer.

Action:
```
{
  "action": "Search",
  "action_input": "year Qatar World Cup took place"
}
```
Observation: 20 Oct 2

"Camila Morrone's current age is 25. Her age raised to the power of 0.43 is approximately 3.991298452658078."